In [1]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import glob
import os

# Conectar ao PostgreSQL (NeonDB)
conn = psycopg2.connect(
    dbname="neondb",  # Substitua com o nome do seu banco de dados
    user="neondb_owner",  # Substitua com o nome de usuário
    password="npg_ayeAjRZ8sPD7",  # Substitua com sua senha
    host="ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech",  # Substitua com o host se necessário
    port="5432"  # Substitua com a porta se necessário
)

try:
    conn = psycopg2.connect(
        dbname="neondb",
        user="neondb_owner",  # Substitua com o nome de usuário
        password="npg_ayeAjRZ8sPD7",  # Substitua com sua senha
        host="ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech",  # Substitua com o host se necessário
        port="5432"
    )
    cur = conn.cursor()
    cur.execute("SELECT version();")
    versao = cur.fetchone()
    print(f"Conexão bem-sucedida! Versão do PostgreSQL: {versao[0]}")
    cur.close()
    conn.close()
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")


Conexão bem-sucedida! Versão do PostgreSQL: PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit


In [2]:
import pandas as pd
import os

# Caminho da pasta contendo os arquivos
folder_path = '/content/drive/MyDrive/06-Base Dados/01-Fluxo_Caixa/03-Nubank/Input'

# Obter lista de arquivos na pasta com a extensão .xls ou .xlsx
arquivos = [f for f in os.listdir(folder_path) if f.endswith('.xls') or f.endswith('.xlsx')]

# Lista para armazenar os DataFrames processados
df_list = []

# Lista fixa com os nomes das colunas
nomes_colunas = [
    'data_lanc', 'categoria', 'descricao', 'valor', 'data_vencimento', 'idcategoria', 'observacao', 'transferencia'
]

# Função para aplicar os nomes fixos nas colunas
def aplicar_nomes_colunas(df, novos_nomes):
    if len(novos_nomes) == len(df.columns):
        df.columns = novos_nomes
    else:
        print(f"Erro: A quantidade de novos nomes não corresponde ao número de colunas em {arquivo}.")
    return df

# Iterar sobre os arquivos da pasta
for arquivo in arquivos:
    # Caminho completo do arquivo
    file_path = os.path.join(folder_path, arquivo)

    # Ler o arquivo Excel (independente de ser .xls ou .xlsx)
    df = pd.read_excel(file_path, header=None)  # Não utiliza um cabeçalho por enquanto

    # Encontrar a linha que contém o cabeçalho (a que tem 'data' na primeira coluna)
    header_row = df[df.iloc[:, 0].str.contains('data', case=False, na=False)].index[0]

    # Ler novamente o arquivo, agora utilizando o cabeçalho correto
    df = pd.read_excel(file_path)
    #df = pd.read_excel(file_path, header=header_row)

    # Aplicar os nomes fixos nas colunas
    df = aplicar_nomes_colunas(df, nomes_colunas)

    # Adicionar uma coluna com o nome do arquivo
    df['nome_arquivo'] = arquivo

    # Adicionar o DataFrame à lista
    df_list.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_combined = pd.concat(df_list, ignore_index=True)

# Ordenar o DataFrame pela coluna 'data'
df_combined['data_lanc'] = pd.to_datetime(df_combined['data_lanc'], errors='coerce', dayfirst=True)  # Converter a coluna 'data' para datetime
df_combined = df_combined.sort_values(by='data_lanc')  # Ordenar pela coluna 'data'

# Adicionar uma coluna 'data_carga' com a data atual
df_combined['data_carga'] = pd.to_datetime('today').normalize()  # 'normalize' para deixar a hora como 00:00:00

colunas_finais = ['data_lanc', 'categoria', 'descricao', 'valor', 'data_vencimento', 'idcategoria', 'observacao', 'nome_arquivo', 'data_carga']

df_filtrado = df_combined[colunas_finais]

# Exibir as primeiras linhas do DataFrame combinado com a nova coluna
#df_filtrado.head()

# exibir a quantidade de linhas para cada arquivo
# print(df_filtrado.groupby('nome_arquivo').size())


,data_lanc,categoria,descricao,valor,data_vencimento,idcategoria,observacao,nome_arquivo,data_carga
0,2022-11-28,vestuário,Yan Calcados 2/3,50.00,2023-01-05,Ves02,calçados,Cartao Nubank.xlsx,2025-04-26
1,2022-11-28,vestuário,Cea Gru 265 Ecpc 2/3,223.97,2023-01-05,Ves03,roupas,Cartao Nubank.xlsx,2025-04-26
2,2022-11-28,vestuário,Pag*Jsgoiscalcados 2/2,39.99,2023-01-05,Ves02,calçados,Cartao Nubank.xlsx,2025-04-26
3,2022-11-28,educação,Pg *Power Bi Pbiexperi 2/5,74.95,2023-01-05,Edu03,curso power bi,Cartao Nubank.xlsx,2025-04-26
4,2022-11-28,educação,Pg *Curso Beta 9/12,59.70,2023-01-05,Edu03,curso beta,Cartao Nubank.xlsx,2025-04-26


In [3]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

# Credenciais do banco de dados
dbname = "neondb"
user = "neondb_owner"
password = "npg_ayeAjRZ8sPD7"
host = "ep-rough-shape-a85bihtu-pooler.eastus2.azure.neon.tech"
port = "5432"

# Criar a string de conexão com o PostgreSQL
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Criar a conexão com o banco de dados
engine = create_engine(connection_string)

In [4]:
from sqlalchemy import text

# Função para excluir dados antigos e inserir novos, com relatórios
def excluir_e_inserir(df, engine):
    with engine.connect() as conn:
        # Verificar se a tabela existe no banco
        check_table_query = text("""
            SELECT EXISTS (
                SELECT FROM information_schema.tables
                WHERE table_schema = 'fluxo'
                AND table_name = 'nubank'
            );
        """)
        table_exists = conn.execute(check_table_query).scalar()

        if table_exists:
            # Se a tabela existir, deletar os dados e gravar novamente
            delete_query = text(f"DELETE FROM fluxo.nubank;")
            conn.execute(delete_query)
            conn.commit()
            df.to_sql('nubank', engine, schema='fluxo', if_exists='append', index=False)
        else:
            # Se a tabela não existir, considera que nenhum arquivo foi importado ainda
            df.to_sql('nubank', engine, schema='fluxo', if_exists='append', index=False)

    # Relatório de execução
    print(f"Processamento concluído.")

# Chamar a função para excluir dados antigos e inserir os novos
excluir_e_inserir(df_filtrado, engine)


Processamento concluído.


In [ ]:
#df_filtrado.info()
#df_filtrado.describe()
#df_filtrado['num_doc'].unique()

#df_filtrado['num_doc'].duplicated().any()


df_filtrado[df_filtrado['num_doc'].duplicated(keep=False)]


,data_lanc,descricao,num_doc,valor,idcategoria,observacao,nome_arquivo,data_carga
0,2019-02-25,Saldo Anterior,0,140.44,NaN,NaN,BB_extrato.xlsx,2025-03-30
2,2022-03-30,Saldo Anterior,0,20.44,NaN,NaN,BB_extrato.xlsx,2025-03-30
3,2022-03-31,S A L D O,0,20.44,NaN,NaN,BB_extrato.xlsx,2025-03-30
5,2022-04-22,Pix - Enviado,42201,-550.00,Outr03,não identificado,BB_extrato.xlsx,2025-03-30
6,2022-04-22,Resgate Poupança,148,342.36,Rec09,resgate Poupança,BB_extrato.xlsx,2025-03-30
...,...,...,...,...,...,...,...,...
1135,2025-03-18,Pix - Enviado - 18/03 16:02 Ana Paula Rodrigue...,31801,-10.32,Outr03,NaN,BB_extrato.xlsx,2025-03-30
1136,2025-03-18,Pix - Enviado - 18/03 19:14 Organizacao F N Ltda,31802,-166.59,Outr03,NaN,BB_extrato.xlsx,2025-03-30
1137,2025-03-20,Pix - Enviado - 20/03 12:31 Drogaria Tranquili...,32001,-178.00,CPes06,NaN,BB_extrato.xlsx,2025-03-30
1145,2025-03-27,Pix - Enviado - 27/03 22:19 Maria Fatima Santo...,32701,-10.00,Outr03,NaN,BB_extrato.xlsx,2025-03-30
